In [1]:
import re
import pickle
import string
import copy
from pathlib import Path

import pandas as pd
import numpy as np
import spacy
import requests
import feedparser
from bs4 import BeautifulSoup

In [37]:
path = 'https://www.cisa.gov/uscert/ncas/alerts.xml'
data_path = Path("../data")

In [3]:
d = feedparser.parse(path)
d['feed']['title']

'CISA Alerts'

In [4]:
d.feed.title

'CISA Alerts'

In [5]:
d.feed.link

'https://us-cert.cisa.gov/'

In [6]:
d.feed.description

'Alerts warn about vulnerabilities, incidents, and other security issues that pose a significant risk.'

In [7]:
d.entries[0].title

'AA22-321A: #StopRansomware: Hive Ransomware'

In [8]:
d.entries[0].link

'https://us-cert.cisa.gov/ncas/alerts/aa22-321a'

In [9]:
d.entries[0].description[:100]

'Original release date: November 17, 2022 | Last revised: November 25, 2022<br /><h3>Summary</h3><p c'

In [10]:
d.entries[0].published

'Thu, 17 Nov 2022 17:00:00 +0000'

In [11]:
d.entries[0].published_parsed

time.struct_time(tm_year=2022, tm_mon=11, tm_mday=17, tm_hour=17, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=321, tm_isdst=0)

In [12]:
d.entries[0].id

'18160 at https://us-cert.cisa.gov'

# get alerts beautifulsoup

In [13]:
import requests
from bs4 import BeautifulSoup

In [14]:
url='https://www.cisa.gov/uscert/ncas/alerts'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [15]:
soup.title

<title>Alerts | CISA</title>

In [16]:
alert_url='https://www.cisa.gov/uscert/ncas/alerts/aa22-321a'
alert_page = requests.get(alert_url)
alert_soup = BeautifulSoup(alert_page.text, 'html.parser')
soup.title

<title>Alerts | CISA</title>

In [17]:
%%time
#get content for each alert on the first 10 pages of alerts
alerts_new = []
soup_alerts_path = Path('soup_alerts.pkl')
if soup_alerts_path.exists():
    with open(soup_alerts_path, "rb") as sap:
        alerts = pickle.load(sap)
else:
    # I think there's only 8 or so pages
    for p_num in range(10): #look at 10 pages of results
        #get the the content from this page
        url='https://www.cisa.gov/uscert/ncas/alerts?page=' + str(p_num)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        table = soup.find_all(class_="views-field views-field-title")

        for i in range(len(table)):
            #get the alert details from this line in the list
            link = table[i].find('a')['href']
            alert_id = table[i].find(class_="field-content").contents[0]
            title = table[i].find('a').text

            #get alert content
            alert_url='https://www.cisa.gov/uscert' + link
            alert_page = requests.get(alert_url)
            alert_soup = BeautifulSoup(alert_page.text, 'html.parser')
            alert_date = alert_soup.find(id="ncas-header").find(class_="submitted meta-text").text
            alert_text = alert_soup.find(id="ncas-content").text

            #write the content
            alerts_new.append([link, alert_id, title, alert_date, alert_text])

    with open(soup_alerts_path, "wb") as sap:
        pickle.dump(alerts_new, sap)
    alerts = alerts_new
alerts[0]

CPU times: user 39.5 ms, sys: 6.65 ms, total: 46.2 ms
Wall time: 46.1 ms


['/ncas/alerts/aa22-321a',
 'AA22-321A : ',
 '#StopRansomware: Hive Ransomware',
 '\n                                Original release date: November 17, 2022  | Last revised: November 25, 2022\n                                            ',
 '\n\nSummary\nActions to Take Today to Mitigate Cyber Threats from Ransomware:\n• Prioritize remediating known exploited vulnerabilities.\n• Enable and enforce multifactor authentication with strong passwords\n• Close unused ports and remove any application not deemed necessary for day-to-day operations.\nNote: This joint Cybersecurity Advisory (CSA) is part of an ongoing #StopRansomware effort to publish advisories for network defenders that detail various ransomware variants and ransomware threat actors. These #StopRansomware advisories include recently and historically observed tactics, techniques, and procedures (TTPs) and indicators of compromise (IOCs) to help organizations protect against ransomware. Visit stopransomware.gov to see all #Stop

In [18]:
%%time
headers = ['link', 'alert_id', 'title', 'date', 'text']
df = pd.DataFrame(alerts, columns=headers)
df.head()

CPU times: user 772 µs, sys: 0 ns, total: 772 µs
Wall time: 772 µs


,link,alert_id,title,date,text
0,/ncas/alerts/aa22-321a,AA22-321A :,#StopRansomware: Hive Ransomware,\n Original rel...,\n\nSummary\nActions to Take Today to Mitigate...
1,/ncas/alerts/aa22-320a,AA22-320A :,Iranian Government-Sponsored APT Actors Compro...,\n Original rel...,\n\nSummary\nFrom mid-June through mid-July 20...
2,/ncas/alerts/aa22-294a,AA22-294A :,#StopRansomware: Daixin Team,\n Original rel...,\n\nSummary\nActions to take today to mitigate...
3,/ncas/alerts/aa22-279a,AA22-279A :,Top CVEs Actively Exploited By People’s Republ...,\n ...,\n\nSummary\nThis joint Cybersecurity Advisory...
4,/ncas/alerts/aa22-277a,AA22-277A :,Impacket and Exfiltration Tool Used to Steal S...,\n Original rel...,\n\nSummary\nActions to Help Protect Against A...


# parse alert content

In [19]:
re_cve = re.compile(r'(CVE-\d{4}-\d+)')
re_cve.findall(alerts[0][4])

['CVE-2020-12812',
 'CVE-2021-31207',
 'CVE-2021-34473',
 'CVE-2021-34523',
 'CVE-2021-34473',
 'CVE-2021-34523',
 'CVE-2021-31207',
 'CVE-2021-42321']

In [20]:
re_ttp = re.compile(r'(\[|\s)(T\d{4})\.?\d*(\]|\s)')
[ttp[1] for ttp in re_ttp.findall(alerts[1][4])]

['T1190',
 'T1059',
 'T1562',
 'T1105',
 'T1070',
 'T1136',
 'T1016',
 'T1053',
 'T1021',
 'T1078',
 'T1136',
 'T1090',
 'T1018',
 'T1098',
 'T1003',
 'T1190',
 'T1059',
 'T1098',
 'T1136',
 'T1136',
 'T1053',
 'T1078',
 'T1562',
 'T1070',
 'T1003',
 'T1555',
 'T1018',
 'T1016',
 'T1021',
 'T1090',
 'T1105']

In [21]:
#regex for cve match
re_cve = re.compile(r'(CVE-\d{4}-\d*)')
cves = []
for i in range(len(alerts)):
    alert_cves = sorted(re_cve.findall(alerts[i][4]))
    cves.append(alert_cves)
cves = [set(cve) for cve in cves]

ttps = []
for i in range(len(alerts)):
    alert_ttps = [ttp[1] for ttp in re_ttp.findall(alerts[i][4])]
    alert_ttps = sorted(alert_ttps)
    ttps.append(alert_ttps)
ttps = [set(ttp) for ttp in ttps]

In [22]:
#add list of CVE & TTP to df
df['CVE_list'] = cves
df['TTP_list'] = ttps
print(df.describe())
df.head()

                          link      alert_id  \
count                      286           286   
unique                     286           286   
top     /ncas/alerts/aa22-321a  AA22-321A :    
freq                         1             1   

                                                 title  \
count                                              286   
unique                                             210   
top     Microsoft Updates for Multiple Vulnerabilities   
freq                                                56   

                                                     date  \
count                                                 286   
unique                                                277   
top     \n                                Original rel...   
freq                                                    2   

                                                     text CVE_list TTP_list  
count                                                 286      286      286  
unique

,link,alert_id,title,date,text,CVE_list,TTP_list
0,/ncas/alerts/aa22-321a,AA22-321A :,#StopRansomware: Hive Ransomware,\n Original rel...,\n\nSummary\nActions to Take Today to Mitigate...,"{CVE-2021-34523, CVE-2020-12812, CVE-2021-3447...","{T1486, T1070, T1112, T1537, T1133, T1190, T15..."
1,/ncas/alerts/aa22-320a,AA22-320A :,Iranian Government-Sponsored APT Actors Compro...,\n Original rel...,\n\nSummary\nFrom mid-June through mid-July 20...,{CVE-2021-44228},"{T1016, T1070, T1021, T1003, T1098, T1053, T11..."
2,/ncas/alerts/aa22-294a,AA22-294A :,#StopRansomware: Daixin Team,\n Original rel...,\n\nSummary\nActions to take today to mitigate...,{},"{T1486, T1598, T1550, T1567, T1003, T1098, T11..."
3,/ncas/alerts/aa22-279a,AA22-279A :,Top CVEs Actively Exploited By People’s Republ...,\n ...,\n\nSummary\nThis joint Cybersecurity Advisory...,"{CVE-2021-36260, CVE-2021-44228, CVE-2021-4201...",{}
4,/ncas/alerts/aa22-277a,AA22-277A :,Impacket and Exfiltration Tool Used to Steal S...,\n Original rel...,\n\nSummary\nActions to Help Protect Against A...,"{CVE-2021-26857, CVE-2021-26858, CVE-2021-2685...","{T1082, T1574, T1033, T1039, T1036, T1059, T10..."


# NER

In [23]:
# Some cleaning up of text field before NER
strings_to_replace= ["Summary", ("\n","  "),("\t","  "), re.compile(r"DISCLAIMER.*"), re.compile(r"INFORMATION REQUESTED.*")]
text = copy.deepcopy(df['text'])

def replace_non_printable(raw_text):
    filtered_text = list(filter(lambda x: x in string.printable, raw_text))
    return "".join(filtered_text)

for str_pattern in strings_to_replace:
    new_text = " "
    old_text = str_pattern
    if type(str_pattern) is tuple:
        new_text = str_pattern[1]
        old_text = str_pattern[0]
    
    if isinstance(str_pattern,str):
        text = text.str.replace(old_text,new_text).str.strip()
    else:
        text = text.str.replace(old_text,new_text,regex=True).str.strip()

# Remove Nonprintable, CVE IDs, and TTPs
text = text.apply(lambda raw_x: replace_non_printable(raw_x))
text = text.str.replace(re_cve,"",regex=True)
text = text.str.replace(re_ttp,"",regex=True)

df['text2'] = text
text[0]

'Actions to Take Today to Mitigate Cyber Threats from Ransomware:   Prioritize remediating known exploited vulnerabilities.   Enable and enforce multifactor authentication with strong passwords   Close unused ports and remove any application not deemed necessary for day-to-day operations.  Note: This joint Cybersecurity Advisory (CSA) is part of an ongoing #StopRansomware effort to publish advisories for network defenders that detail various ransomware variants and ransomware threat actors. These #StopRansomware advisories include recently and historically observed tactics, techniques, and procedures (TTPs) and indicators of compromise (IOCs) to help organizations protect against ransomware. Visit stopransomware.gov to see all #StopRansomware advisories and to learn more about other ransomware threats and no-cost resources.  The Federal Bureau of Investigation (FBI), the Cybersecurity and Infrastructure Security Agency (CISA), and the Department of Health and Human Services (HHS) are r

In [26]:
# Clean alert id
df['alert_id'] = df['alert_id'].str.replace(' :','').str.strip()

# Extract date from date field
df['date'] = pd.to_datetime(df['date'].str.replace("Original release date: ","",regex=False).str.replace("\|.*","",regex=True).str.strip())

#convert link to full path
df['full_path'] = 'https://www.cisa.gov' + df['link']


In [24]:
nlp = spacy.load("en_core_web_trf")

In [ ]:
%%time
# Extract entity from each text2
ner_pickle_path = data_path / "ner_text2.pkl"
if 'ner2' in df.columns:
    print("Not running the entity recognition, since the data already exists as 'ner' in the dataframe.")
elif ner_pickle_path.exists():
    with open(ner_pickle_path, "rb") as npp:
        df["ner2"] = pickle.load(ner_pickle_path)
else:
    ner2 = []
    for i in range(len(df['text2'])):
        #nlp this doc
        this_doc = nlp(df['text2'][i])

        #select get entities from this doc
        this_ner = []
        for ent in this_doc.ents:
            #choose from list of 
            ent_labels = ['ORG','PRODUCT','GPE','PERSON','NORP']
            if ent.label_ in ent_labels:
                this_ner.append([ent.text, ent.label_])

        #append ner for this doc
        ner2.append(this_ner)
    with open(ner_pickle_path, "wb") as npp:
        pickle.dump(ner2, npp)
    df['ner2'] = ner2
df['ner2']

In [28]:
%%time
#extract entity from each text
ner_pickle_path = Path("alerts") / "ner.pkl"
if 'ner' in df.columns:
    print("Not running the entity recognition, since the data already exists as 'ner' in the dataframe.")
elif ner_pickle_path.exists():
    with open(ner_pickle_path, "rb") as npp:
        df['ner'] = pickle.load(ner_pickle_path)
else:
    ner = []
    for i in range(len(alerts)):
        #nlp this doc
        this_doc = nlp(alerts[i][4])

        #select get entities from this doc
        this_ner = []
        for ent in this_doc.ents:
            #choose from list of 
            ent_labels = ['ORG','PRODUCT','GPE','PERSON','NORP']
            if ent.label_ in ent_labels:
                this_ner.append([ent.text, ent.label_])

        #append ner for this doc
        ner.append(this_ner)
    with open(ner_pickle_path, "wb") as npp:
        pickle.dump(ner, npp)

    # add ner into df. leave duplicates for other opportunity
    df['ner'] = ner

df['ner']

Not running the entity recognition, since the data already exists as 'ner' in the dataframe.
CPU times: user 50 µs, sys: 12 µs, total: 62 µs
Wall time: 71.3 µs


In [ ]:
%%time
# Extract entity from each text2
ner_pickle_path = data_path / "ner_text2.pkl"
if 'ner2' in df.columns:
    print("Not running the entity recognition, since the data already exists as 'ner' in the dataframe.")
elif ner_pickle_path.exists():
    with open(ner_pickle_path, "rb") as npp:
        df["ner2"] = pickle.load(ner_pickle_path)
else:
    ner2 = []
    for i in range(len(df['text2'])):
        #nlp this doc
        this_doc = nlp(df['text2'][i])

        #select get entities from this doc
        this_ner = []
        for ent in this_doc.ents:
            #choose from list of 
            ent_labels = ['ORG','PRODUCT','GPE','PERSON','NORP']
            if ent.label_ in ent_labels:
                this_ner.append([ent.text, ent.label_])

        #append ner for this doc
        ner2.append(this_ner)
    with open(ner_pickle_path, "wb") as npp:
        pickle.dump(ner2, npp)
    df['ner2'] = ner2
df['ner2']

In [ ]:
# REmove this after running once!
if "ner2" not in locals():
    ner2 = copy.deepcopy(ner)

In [30]:
# Only write pickle if df has values
if df is not None and not df.empty:
    df.to_pickle("./alerts_df.pkl")

In [31]:
# Load in the dataframe if it's not there already
if "df" not in locals():
    df = pd.read_pickle("./alerts_df.pkl")

In [32]:
df.head()

,link,alert_id,title,date,text,CVE_list,TTP_list,text2,full_path,ner
0,/ncas/alerts/aa22-321a,AA22-321A,#StopRansomware: Hive Ransomware,2022-11-17,\n\nSummary\nActions to Take Today to Mitigate...,"{CVE-2021-34523, CVE-2020-12812, CVE-2021-3447...","{T1486, T1070, T1112, T1537, T1133, T1190, T15...",Actions to Take Today to Mitigate Cyber Threat...,https://www.cisa.gov/ncas/alerts/aa22-321a,"[[stopransomware.gov, ORG], [The Federal Burea..."
1,/ncas/alerts/aa22-320a,AA22-320A,Iranian Government-Sponsored APT Actors Compro...,2022-11-16,\n\nSummary\nFrom mid-June through mid-July 20...,{CVE-2021-44228},"{T1016, T1070, T1021, T1003, T1098, T1053, T11...","From mid-June through mid-July 2022, CISA cond...",https://www.cisa.gov/ncas/alerts/aa22-320a,"[[CISA, ORG], [Federal Civilian Executive Bran..."
2,/ncas/alerts/aa22-294a,AA22-294A,#StopRansomware: Daixin Team,2022-10-21,\n\nSummary\nActions to take today to mitigate...,{},"{T1486, T1598, T1550, T1567, T1003, T1098, T11...",Actions to take today to mitigate cyber threat...,https://www.cisa.gov/ncas/alerts/aa22-294a,"[[The Federal Bureau of Investigation, ORG], [..."
3,/ncas/alerts/aa22-279a,AA22-279A,Top CVEs Actively Exploited By People’s Republ...,2022-10-06,\n\nSummary\nThis joint Cybersecurity Advisory...,"{CVE-2021-36260, CVE-2021-44228, CVE-2021-4201...",{},This joint Cybersecurity Advisory (CSA) provid...,https://www.cisa.gov/ncas/alerts/aa22-279a,"[[People’s Republic of China, GPE], [PRC, GPE]..."
4,/ncas/alerts/aa22-277a,AA22-277A,Impacket and Exfiltration Tool Used to Steal S...,2022-10-04,\n\nSummary\nActions to Help Protect Against A...,"{CVE-2021-26857, CVE-2021-26858, CVE-2021-2685...","{T1082, T1574, T1033, T1039, T1036, T1059, T10...",Actions to Help Protect Against APT Cyber Acti...,https://www.cisa.gov/ncas/alerts/aa22-277a,[[the Cybersecurity and Infrastructure Securit...


In [33]:
print(df.iloc[[0],[3]])

        date
0 2022-11-17


# alert to CVE edges

In [38]:
#alert to cve edges
alert_cve = df[['alert_id','CVE_list']].explode('CVE_list')
alert_cve = alert_cve.drop_duplicates().dropna().reset_index(drop=True)
alert_cve.to_csv(data_path / 'alert_cve_edge.csv', index=False)
alert_cve

,alert_id,CVE_list
0,AA22-321A,CVE-2021-34523
1,AA22-321A,CVE-2020-12812
2,AA22-321A,CVE-2021-34473
3,AA22-321A,CVE-2021-31207
4,AA22-321A,CVE-2021-42321
...,...,...
391,TA09-195A,CVE-2008-0015
392,TA09-195A,CVE-2009-1537
393,TA09-133B,CVE-2009-1492
394,TA09-133B,CVE-2009-1493


# alert nodes

In [40]:
alerts_nodes = df[['alert_id','full_path','title','date']].drop_duplicates()
alerts_nodes.to_csv(data_path / 'alert_nodes.csv')
alerts_nodes

,alert_id,full_path,title,date
0,AA22-321A,https://www.cisa.gov/ncas/alerts/aa22-321a,#StopRansomware: Hive Ransomware,2022-11-17
1,AA22-320A,https://www.cisa.gov/ncas/alerts/aa22-320a,Iranian Government-Sponsored APT Actors Compro...,2022-11-16
2,AA22-294A,https://www.cisa.gov/ncas/alerts/aa22-294a,#StopRansomware: Daixin Team,2022-10-21
3,AA22-279A,https://www.cisa.gov/ncas/alerts/aa22-279a,Top CVEs Actively Exploited By People’s Republ...,2022-10-06
4,AA22-277A,https://www.cisa.gov/ncas/alerts/aa22-277a,Impacket and Exfiltration Tool Used to Steal S...,2022-10-04
...,...,...,...,...
281,TA08-352A,https://www.cisa.gov/ncas/alerts/TA08-352A,Microsoft Internet Explorer Data Binding Vulne...,2008-12-17
282,TA08-350A,https://www.cisa.gov/ncas/alerts/TA08-350A,Apple Updates for Multiple Vulnerabilities,2008-12-15
283,TA08-344A,https://www.cisa.gov/ncas/alerts/TA08-344A,Microsoft Updates for Multiple Vulnerabilities,2008-12-09
284,TA08-340A,https://www.cisa.gov/ncas/alerts/TA08-340A,Sun Java Updates for Multiple Vulnerabilities,2008-12-05


# dedupe

In [41]:
import os
import csv
import re
import logging
import optparse

import dedupe
from unidecode import unidecode

In [67]:
#alert to entity edges
df_ner = df[['alert_id','ner']].explode('ner')
df_ner[['label','type']] = pd.DataFrame(df_ner.ner.tolist(), index= df_ner.index)
df_ner=df_ner.reset_index(level=0)
df_ner=df_ner.drop(columns='ner')

In [46]:
ner_dedupe = df_ner[['label', 'type']]
ner_dedupe = ner_dedupe.drop_duplicates()
ner_dedupe = ner_dedupe.reset_index(drop=True)
ner_dedupe.to_csv(data_path / 'alert_ner.csv', index=True, index_label = 'Id', columns=['label','type'])
ner_dedupe

,label,type
0,stopransomware.gov,ORG
1,The Federal Bureau of Investigation,ORG
2,FBI,ORG
3,the Cybersecurity and Infrastructure Security ...,ORG
4,CISA,ORG
...,...,...
3632,Mozilla Foundation,ORG
3633,Firefox 3.0.4,PRODUCT
3634,Firefox 2.0.0.18,PRODUCT
3635,SeaMonkey 1.1.13,PRODUCT


In [49]:
#Do a little bit of data cleaning with the help of Unidecode and Regex. Things like casing, extra spaces, quotes and new lines can be ignored.

def preProcess(column):
#
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
#If data is missing, indicate that by setting the value to None

    if not column:
        column = None
    return column
#Read in our data from a CSV file and create a dictionary of records, where the key is a unique record ID and each value is dict

def readData(filename):
#
    data_d = {}
    with open(filename, encoding="utf8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d


In [54]:
input_file = data_path / 'alert_ner.csv'
output_file = data_path / 'ner_cluster.csv'
training_file = data_path / 'ner_training.json'

In [50]:
data_d=readData(input_file)

In [52]:
#define dedupe
fields = [
            {'field': 'label', 'type': 'String'},            {'field': 'type', 'type': 'String'}
            ]
deduper = dedupe.Dedupe(fields)

In [53]:
print('preparing training...')
if os.path.exists(training_file):
    print('reading labeled examples from ', training_file)
    with open(training_file, 'rb') as f:
        deduper.prepare_training(data_d, f)
else:
    deduper.prepare_training(data_d)

preparing training...


In [55]:
print('starting active labeling...')
dedupe.console_label(deduper)

label : the cybersecurity and infrastructure security agency
type : org

label : the cybersecurity and infrastructure security agency's
type : org

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


starting active labeling...


 y


label : the department of the treasury
type : org

label : the department of the treasury's
type : org

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 y


...

label : u.s. department of justice
type : org

label : the department of justice
type : org

83/10 positive, 98/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 y


label : arkansas
type : gpe

label : argentinabelgiumbrazilcambodiachinacolombiaegyptindiairanjordanpakistansaudi
type : gpe

84/10 positive, 98/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 f


Finished labeling


In [56]:
deduper.train()

with open(training_file, 'w') as tf:
        deduper.write_training(tf)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [57]:
print('clustering...')
clustered_dupes = deduper.partition(data_d, 0.5)

print('# duplicate sets', len(clustered_dupes))

clustering...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/to

In [58]:
cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            "Cluster ID": cluster_id,
            "confidence_score": score
        }

with open(output_file, 'w') as f_output, open(input_file, encoding="utf8") as f_input:

    reader = csv.DictReader(f_input)
    fieldnames = ['Cluster ID', 'confidence_score'] + reader.fieldnames

    writer = csv.DictWriter(f_output, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        row_id = int(row['Id'])
        row.update(cluster_membership[row_id])
        writer.writerow(row)

# alert to ner edge

In [66]:
df_clusters = pd.read_csv(data_path / 'ner_cluster.csv')

** TODO **
1. Remove punctuation from labels
2. Remove any labels that have only < 2 characters once punctuation is removed
3. Plot the confidence scores and check accuracy around the chosen threshold
4. Figure out why so many matches are not correct. "stopransomware" matching with CISA, FBI doesn't make sense, same with "MS Windows and Firefox, Thunderbird, SeaMonkey.
**5. Match "PRODUCT" type entities to CPE list**



In [68]:
df_ner_clusters = df_ner.merge(df_clusters, left_on='index', right_on='Id', suffixes=["_x","_y"]).drop(columns="index")
df_ner_clusters

,alert_id,label_x,type_x,Cluster ID,confidence_score,Id,label_y,type_y
0,AA22-321A,stopransomware.gov,ORG,1,0.800972,0,stopransomware.gov,ORG
1,AA22-321A,The Federal Bureau of Investigation,ORG,1,0.800972,0,stopransomware.gov,ORG
2,AA22-321A,FBI,ORG,1,0.800972,0,stopransomware.gov,ORG
3,AA22-321A,the Cybersecurity and Infrastructure Security ...,ORG,1,0.800972,0,stopransomware.gov,ORG
4,AA22-321A,CISA,ORG,1,0.800972,0,stopransomware.gov,ORG
...,...,...,...,...,...,...,...,...
18405,TA08-319A,Thunderbird,PRODUCT,794,1.000000,285,MS Windows,PRODUCT
18406,TA08-319A,SeaMonkey 1.1.13,PRODUCT,794,1.000000,285,MS Windows,PRODUCT
18407,TA08-319A,Mozilla Foundation,ORG,794,1.000000,285,MS Windows,PRODUCT
18408,TA08-319A,Mozilla,ORG,794,1.000000,285,MS Windows,PRODUCT


In [76]:
alert_ner_edge = pd.DataFrame(df_ner_clusters.groupby(by=['alert_id','label_x']).size())
alert_ner_edge = alert_ner_edge.reset_index()
alert_ner_edge.to_csv(data_path / 'alert_ner_edge.csv', index=False, header=['alert_id','ner','weight'])
alert_ner_edge

,alert_id,label_x,0
0,AA18-284A,ACSC,2
1,AA18-284A,APT19,2
2,AA18-284A,Active Directory,1
3,AA18-284A,Adobe,1
4,AA18-284A,Adwind RAT,2
...,...,...,...
7230,TA18-331A,Windows,1
7231,TA18-331A,YARA,1
7232,TA18-331A,the Department of Homeland Security,1
7233,TA18-331A,the Federal Bureau of Investigation,1


# ner node

In [74]:
ner_node = df_ner_clusters.groupby(by=['label_x','Cluster ID']).size()
ner_node = ner_node.reset_index()
ner_node.to_csv(data_path / 'ner_node.csv', index=False, header=['ner','cluster id','weight'])
ner_node

,label_x,Cluster ID,0
0,!,744,1
1,%AppData% folder,712,1
2,-,7,2
3,-,47,1
4,-,75,1
...,...,...,...
7097,workstation,58,1
7098,x64 Edition Service Pack,751,2
7099,x86,62,1
7100,yum,44,1
